In [1]:
from pyrpipe import sra,mapping,assembly,qc,tools
#First get the srr accessions of the runs. For this one can use the python package pysradb or R package sradb
#runs=['SRR3098746','SRR3098745','SRR3098744'] #from the study SRP068369
runs=['SRR765545'] #small test
#set up directories
maize_data="/home/usingh/work/urmi/hoap/test/maize/maize_data"
workingDir="/home/usingh/work/urmi/hoap/test/maize"



Logs will be saved to 2020-01-16-16_48_04_pyrpipe.log


## Download data, pre-process

In [2]:
sraObjects=[]
for x in runs:
    thisSraOb=sra.SRA(x,workingDir)
    if thisSraOb.download_sra():
        sraObjects.append(thisSraOb)
    else:
        print("Download failed:"+x)
        
#perform fastq dump and qc

#create a Trimgalore object
tgOpts={"--cores": "10"}
tg=qc.Trimgalore(**tgOpts)

for x in sraObjects:
    #to fastq
    x.run_fasterqdump(delete_sra=True,**{"-e":"20","-f":"","-t":workingDir}) #use 20 threads
    #perform qc using trim galore
    x.perform_qc(tg)
    

        

$ prefetch -O /home/usingh/work/urmi/hoap/test/maize/SRR765545 SRR765545
Time taken:0:00:14
Downloaded file: /home/usingh/work/urmi/hoap/test/maize/SRR765545/SRR765545.sra 500.4 MB 
$ fasterq-dump -e 20 -f -t /home/usingh/work/urmi/hoap/test/maize -O /home/usingh/work/urmi/hoap/test/maize/SRR765545 -o SRR765545.fastq /home/usingh/work/urmi/hoap/test/maize/SRR765545/SRR765545.sra
Time taken:0:00:27
Performing QC using trim_galore
$ trim_galore --cores 10 --paired -o /home/usingh/work/urmi/hoap/test/maize/SRR765545 /home/usingh/work/urmi/hoap/test/maize/SRR765545/SRR765545_1.fastq /home/usingh/work/urmi/hoap/test/maize/SRR765545/SRR765545_2.fastq
Time taken:0:01:09


## Map using STAR

In [ ]:
starParams={"--outFilterType":"BySJout",
            "--runThreadN":"8",
            "--outSAMtype": "BAM SortedByCoordinate"
            }

star=mapping.Star(star_index="",**starParams) #provided index is invalid

#create star index
indexOut=maize_data+"/starindex"
inFasta=maize_data+"/Zea_mays.B73_RefGen_v4.dna.toplevel.1_10.fa"
star.build_index(indexOut,inFasta)

#after building index run star
#star_out=[]
#for x in sraObjects:
#    star_out.append(star.perform_alignment(x))

No STAR index provided. Please build index now to generate an index using build_index()....
$ STAR --runMode genomeGenerate --genomeDir /home/usingh/work/urmi/hoap/test/maize/maize_data/starindex --genomeFastaFiles /home/usingh/work/urmi/hoap/test/maize/maize_data/Zea_mays.B73_RefGen_v4.dna.toplevel.1_10.fa
